In [ ]:
%cd CPMP-ML

## Librerias Necesarias

In [ ]:
import numpy as np
import pandas as pd
import keras
from keras.layers import Input, Add, Dense, MultiHeadAttention, LayerNormalization, Dropout, Flatten
from keras.models import Model, load_model
from keras.utils import to_categorical
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.metrics import accuracy_score
import cpmp_ml
import numpy as np
from cpmp_ml import generate_random_layout
import tensorflow as tf
from copy import deepcopy
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
%cd ..

# Funciones necesarias

In [ ]:
#************* | Función is_valid_BG_move() | ****************#
# El proposito de esta función es validar si el movimiento    #
# dentro del estado actual del problema actual al que se le   #
# desea realizar sirve para resolver el problema o no.        #
#                                                             #
# Input:                                                      #
#     - layout: Variable que contiene el estado al que se     #
#               le desea realizar el movimiento.              #
#     - s_o: Stack de origen                                  #
#     - S_d: Stack de destino                                 #
#                                                             #
# Output:                                                     #
#     La función retorna True en caso de ser un movimiento    #
#     posible o False en caso contrario.                      #
def is_valid_BG_move(layout, s_o, s_d):
    if (s_o != s_d  and len(layout.stacks[s_o]) > 0
            and  len(layout.stacks[s_d]) < layout.H
            and layout.is_sorted_stack(s_o)==False
            and layout.is_sorted_stack(s_d)==True
            and layout.gvalue(s_o) <= layout.gvalue(s_d)):
        return True

    else: return False

In [ ]:
#*********** | Función select_bg_move() | ************#
# El proposito de esta función es seleccionar un      #
# movimiento que optimice la diferencia entre las     #
# prioridades entre los contenedores del stack de     #
# destino y el stack de origen.                       #
#                                                     #
# Input:                                              #
#     - layout: Variable que contiene el estado       #
#               actual al que se le desea realizar    #
#               el movimiento.                        #
#                                                     #
# Output:                                             #
#   Retorna una tupla indicando el stack de origen    #
#   y el stack de destino que minimiza la             #
#   diferencia antes mencionada o en caso contrario   #
#   retorna None.                                     #
def select_bg_move(layout):
    bg_move = None # Tupla de movimiento optimo
    S=len(layout.stacks) # Cantidad de stacks
    min_diff = 100
    for s_o in range(S):
        for s_d in range(S):
            if is_valid_BG_move(layout, s_o, s_d):
                diff = layout.gvalue(s_d) - layout.gvalue(s_o)
                if min_diff > diff:
                    min_diff = diff
                    bg_move = (s_o,s_d)
    return bg_move

In [ ]:
#************* | Función greedy | **************#
# El proposito de esta función es utilizar un   #
# algoritmo greedy para resolver el problema    #
# CPMP.                                         #
#                                               #
# Input:                                        #
#     - layout: El estado al cual se le desea   #
#               aplicar el algoritmo greedy     #
#               para resolverlo.                #
# Output:                                       #
#     La función retorna la cantidad de         #
#     movimientos que toma resolver el problema #
#     con el algoritmo y los movimientos que    #
#     debe llevar a cabo, en caso de no poder   #
#     ser resuelto retorna como cantidad de     #
#     pasos -1 y None en movimientos.           #
def greedy(layout):
    moves = []
    steps = 0
    while layout.unsorted_stacks>0:
        bg_move=select_bg_move(layout)
        if bg_move is not None:
            moves.append(bg_move)
            layout.move(bg_move)
        else:
            return -1, None # no lo resuelve
        steps +=1

    if layout.unsorted_stacks==0:
        return steps, moves
    return -1, None

In [ ]:
#**************** | Función unlist() | ***************#
# El proposito de esta función es unificar las        #
# diversas listas de datos con los nuevos datos a     #
# ingresar.                                           #
#                                                     #
# Input:                                              #
#     - data: Lista de matrices principal.            #
#     - labels1: Lista de stacks de origen principal. #
#     - labels2: Lista de stacks de destino principal.#
#     - state: Lista de matrices con nuevos datos.    #
#     - source: Lista de stacks de origen con nuevos  #
#               datos.                                #
#     - destination: Lista de stacks de destino con   #
#                    nuevos datos.                    #
#                                                     #
# Output:                                             #
#     No retorna valores.                             #
def unlist(data,labels1, labels2, state, source, destination):
    n = len(source) # Cantidad de datos nuevos
    for i in range(n):
        data.append(state[i])
        labels1.append(source[i])
        labels2.append(destination[i])

In [ ]:
#************* | get_ann_state() | **************#
# El proposito de esta función preparar los      #
# datos de un estado del problema CPMP para que  #
# pueda ser leído por una red neuronal.          #
#                                                #
# Input:                                         #
#     - layout: Estado actual del problema CPMP. #
#                                                #
# Output:                                        #
#     Retorna una matriz con los datos           #
#     normalizados.                              #
def get_ann_state(layout):
    S=len(layout.stacks) # Cantidad de stacks
    #matriz de stacks
    b = 2. * np.ones([S,layout.H + 1]) # Matriz normalizada
    for i,j in enumerate(layout.stacks):
        b[i][layout.H-len(j) + 1:] = [k/layout.total_elements for k in j]
        b[i][0] = layout.is_sorted_stack(i)
    b.shape=(S,(layout.H + 1))
    return b

#overriding the function in the module
cpmp_ml.get_ann_state = get_ann_state

In [ ]:
#************ | convert_label() | ************#
# El proposito de esta función es convertir   #
# una matriz en una lista con los datos       #
# apilados de forma que una red neuronal los  #
# use como etiquetas.                         #
#                                             #
# Input:                                      #
#     - score: Una matriz con datos binarios. #
#                                             #
# Output:                                     #
#      Retorna una lista con los datos de la  #
#      matriz apilados.                       #
def convert_label(score):
    list_score = []

    for i in score:
        list_score.append(i)

    return np.array(list_score)

In [ ]:
#************ | generate_steps_of_a_state() | *****************#
# El proposito de esta función es generar todos los            #
# movimientos que resuelven un estado a través de un algorimo  #
# greedy.                                                      #
#                                                              #
# Input:                                                       #
#     - layout: El estado actual que se desea resolver.        #
#                                                              #
# Output:                                                      #
#      - states: Lista de todos los estados de la solución.    #
#      - source_tags: Lista de todos los stacks de origen.     #
#      - destination_tags: Lista de todos los stacks de        #
#                          destino.                            #
#       En caso de no poder resolver el estado actual,         #
#       cada una de las variables antes mencionadas retorna    #
#       None.                                                  #
def generate_steps_of_a_state(layout):
    states = [] # estados antes de movimiento
    source_tags = [] # etiquetas origen
    destination_tags = []  # etiquetas destino

    aux = deepcopy(layout)  # creación de copia de objeto

    steps, moves = greedy(aux)
    if steps == -1:
        return None, None, None

    for i in range(steps):
        states.append(get_ann_state(layout)) # antes del movimiento
        source_tags.append(convert_label(to_categorical(moves[i][0],len(layout.stacks))))
        destination_tags.append(convert_label(to_categorical(moves[i][1],len(layout.stacks))))
        layout.move(moves[i])
        aux = deepcopy(layout)

    return states, source_tags, destination_tags

In [ ]:
#*************** | create_different_states() | ***************#
# El proposito de esta función es crear diferentes            #
# estados para el problema CPMP con sus respectivas           #
# soluciones calculadas con un algoritmo greedy.              #
#                                                             #
# Input:                                                      #
#     - S: Cantidad de stacks.                                #
#     - H: Altura de los stacks                               #
#     - max_priority_container: la prioridad máxima de        #
#                               los contenedores y a la       #
#                               vez el total de contenedores. #
#     - n: cantidad de nuevos estados.                        #
# Output:                                                     #
#     Retorna arreglos que representan los distintos          #
#     estados con otros dos arreglos que contienen sus        #
#     posibles stacks de origen y de destino.                 #
def create_different_states(S, H, max_priority_container, n):
    data_state = []
    labels_source = []
    labels_destination = []
    iter = 0
    while iter < n:
        layout = generate_random_layout(S,H,max_priority_container)
        state, source, destination = generate_steps_of_a_state(layout)
        if state == None and source == None and destination == None: continue
        unlist(data_state,labels_source,labels_destination,state,source,destination)
        iter+=1

    return np.stack(data_state), np.stack(labels_source), np.stack(labels_destination)

In [ ]:
#***************** | save_data() | *****************#
# El proposito de esta función es almacenar datos   #
# de entrenamiento en un archivo csv.               #
#                                                   #
# Input:                                            #
#     - data: Lista de matrices con los estados     #
#             correspondientes.                     #
#     - labels_1: Lista con los posibles stacks de  #
#                 origen.                           #
#     - labels_2: Lista con los posibles stacks de  #
#                 destino.                          #
#     - name: Dirección y/o nombre del archivo      #
#             sin el .csv.                          #
# Output:                                           #
#      No retorna valores.                          #
def save_data(data, labels_1, labels_2, name):
    size_stack = len(data[0])
    size_height = len(data[0][0])
    total_data = len(data)
    tuple_data = zip(data, labels_1, labels_2)

    with open(name + '.csv', 'w') as archivo:
        archivo.write('Total data: ' + str(total_data) + '\n')
        archivo.write('Stacks:' + str(size_stack) + ',Height:' + str(size_height) + '\n')
        archivo.write('\n')

        for matrix, label_1, label_2 in tuple_data:
            lista_stack = matrix.flatten()
            archivo.write('matrix:' + str(lista_stack.tolist())[1:-1] + '\n')
            archivo.write('label_1:' + str(label_1.tolist())[1:-1] + '\n')
            archivo.write('label_2:' + str(label_2.tolist())[1:-1] + '\n')
            archivo.write('\n')

        archivo.close()

In [ ]:
#************** | load_data() | ************#
# El proposito de esta función es cargar    #
# los datos almacenados en un csv para el   #
# entrenamiento de la red neuronal.         #
#                                           #
# Input:                                    #
#     - name: Nombre del archivo con su     #
#             dirección correspondiente     #
#             sin el .csv.                  #
#                                           #
# Output:                                   #
#      Retorna 3 arreglos, el primero       #
#      corresponde a una lista de estados,  #
#      el segundo a una lista de labels     #
#      de stacks de origen y por último una #
#      lista de posibles stacks de destino. #
def load_data(name):
    data = []
    labels_1 = []
    labels_2 = []

    with open(name + '.csv', 'r') as archivo:
        total = int(archivo.readline().split(':')[1])
        line = archivo.readline().split(':')
        size_stacks = int(line[1].split(',')[0])
        size_height = int(line[2])
        archivo.readline()

        for i in range(total):
            matrix = archivo.readline().split(':')[1].split(',')
            matrix = np.array(matrix, dtype= float)
            matrix = np.reshape(matrix, (size_stacks, size_height))

            label_1 = archivo.readline().split(':')[1].split(',')
            label_1 = np.array(label_1, dtype= float)
            label_2 = archivo.readline().split(':')[1].split(',')
            label_2 = np.array(label_2, dtype= float)

            data.append(matrix)
            labels_1.append(label_1)
            labels_2.append(label_2)

            archivo.readline()

    return np.stack(data), np.stack(labels_1), np.stack(labels_2)

# Generador de data

In [ ]:
# Cantidad de stacks
S = 5#@param {type:'slider',min:1,max:1000,steps:1}

# Altura de la bahía
H = 7#@param {type:'slider',min:1,max:1000,steps:1}

# Número máximo de prioridad
MPC = 32 #@param {type:'slider',min:1,max:1000,steps:1}

# Cantida casos de entrenamiento
N = 10000 #@param {type:'slider',min:1,max:100000,steps:1}

X_train, y_train_1, y_train_2 = create_different_states(S, H, MPC, N)

In [ ]:
# Cantidad de stacks
S2 = 3 #@param {type:'slider',min:1,max:100,steps:1}

# Altura de la bahía
H2 = 7 #@param {type:'slider',min:1,max:100,steps:1}

# Número máximo de prioridad
MPC2 = 12 #@param {type:'slider',min:1,max:100,steps:1}

# Cantida casos de prueba
N2 = 500 #@param {type:'slider',min:1,max:10000,steps:1}

X_test, y_test_1, y_test_2 = create_different_states(S2, H2, MPC2, N2)